In [1]:
import numpy as np
from numpy.linalg import norm
import time
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.optimize import minimize, root
from cvxopt import matrix, solvers
from multiprocessing import Pool

solvers.options['show_progress'] = False

In [2]:
class Segment:

    def __init__(self, a_range, b, n, share):
        self.a = np.random.uniform(a_range[0], a_range[1], size=n)
        self.b = b
        self.share = share
        
    def compute_bounds_x(self, p_lb, p_ub):
        self.x_lb = 1 / (1 + np.sum(np.exp(self.a - self.b * p_lb)))
        self.x_ub = 1 / (1 + np.sum(np.exp(self.a - self.b * p_ub)))
     
    
class Cube:
    
    def __init__(self, center, radius, theta_start=None):
        
        self.feasible = False
        self.remove = False
        self.center = center
        self.bottomcorner = center - radius
        self.radius = radius
        self.theta_start = theta_start
        self.x_delta = None
        self.r_delta = None
        self.D = None
        self.dDdx_norm_ub = None
        self.rev_ub = None
        
        
class BranchAndBound:
    '''
    BrandAndBound description 
    
    '''
    
    def __init__(self, n, m, seed, max_iter, a_range, b_range, epsilon):
        
        self.iter = 0
        self.n = n
        self.m = m
        self.seed = seed
        self.max_iter = max_iter
        self.a_range = a_range
        self.b_range = b_range
        self.epsilon = epsilon
        
        if self.seed:
            np.random.seed(self.seed)
        
        self.w = np.random.uniform(0, 1, size=m)
        self.w /= np.sum(self.w)
        self.b = np.random.uniform(b_range[0], b_range[1], size=n)
        self.segments = [Segment(a_range, self.b, n, wc) for wc in self.w]

        self.radius = None
        self.timer = None
        self.removed_cubes = []
        
        self.omega = np.vstack(
            map(np.ravel, np.meshgrid(*([[-1,1] for _ in range(self.m)])))
        ).T

        self.compute_bounds_p_x()
        self.z_lb = np.exp(-self.p_ub * self.b)
        self.z_ub = np.exp(-self.p_lb * self.b)
        self.x_ub = np.asarray([segment.x_ub for segment in self.segments])
        self.x_lb = np.asarray([segment.x_lb for segment in self.segments])
        self.radius = np.max(self.x_ub - self.x_lb) / 2
        self.E = np.asarray([np.exp(segment.a) for segment in self.segments]).T
        self.k = self.E * self.w.reshape(1,-1) / self.b.reshape(-1,1)
        
        self.xlog = {'cubes': [],
                     'cubes_to_plot': [],
                     'cube_count': [],
                     'rev_lb': [],
                     'opt_gap': []}
    
    
    @staticmethod
    def pi_c(pi, segment, b):
        return pi - np.sum(1 / b * np.exp(segment.a - 1 - b * pi))

    
    @staticmethod
    def rev(p, segments):
        return - np.sum([c.share * np.sum(p * np.exp(c.a - c.b * p)) /
                         (1 + np.sum(np.exp(c.a - c.b * p))) for c in segments])
    
    
    @staticmethod
    def D(theta, E, u, m, n, z_lb, z_ub, kx):

        lam = theta[:m]
        exp_nu_lb = np.exp(theta[m:n+m])
        exp_nu_ub = np.exp(theta[n+m:])
        z = np.exp((E.dot(lam) + exp_nu_ub - exp_nu_lb) / kx - 1)
            
        return (np.inner(kx, z)
                - np.inner(lam, u)
                + np.inner(exp_nu_lb, z_lb)
                - np.inner(exp_nu_ub, z_ub))
    
    
    @staticmethod
    def Dgrad(theta, E, u, m, n, z_lb, z_ub, kx):
        lam = theta[:m]
        exp_nu_lb = np.exp(theta[m:n+m])
        exp_nu_ub = np.exp(theta[n+m:])
        z = np.exp((E.dot(lam) + exp_nu_ub - exp_nu_lb) / kx - 1)
        return np.hstack([
            E.T.dot(z) - u,
            (z_lb - z) * exp_nu_lb,
            (z - z_ub) * exp_nu_ub,
        ])
    
    
    def compute_bounds_p_x(self):
    
        # compute upper bound on segment revenues
        ub_pi = []
        for segment in self.segments:
            res = root(self.pi_c, 5.0, args=(segment, self.b))
            if not res.success:
                raise Exception('Root finding failed.')
            else:
                ub_pi.append(res.x)

        # compute upper bound on prices
        self.p_ub = 1 / self.b + np.max(ub_pi)
        self.p_lb = 1 / self.b

        for segment in self.segments:
            segment.compute_bounds_x(self.p_lb, self.p_ub)

            
    def check_feasibility_cube_cvx(self, cube, scale_r=1.0):
        
        m = self.m
        n = self.n
        x = cube.center
        E = self.E
        u = (1 - x) / x
        r = self.radius

        c = matrix(np.hstack([np.zeros(n + 2 * m), np.ones(1)]))

        b_eq = matrix(u)
        A_eq = matrix(np.hstack([E.T,
                                 np.identity(m),
                                 -np.identity(m),
                                 np.zeros((m, 1))]))

        b_ub_1 = np.zeros(m)
        A_ub_1 = np.hstack([np.zeros((m, n)),
                            np.identity(m),
                            np.identity(m),
                            -np.ones((m,1))])

        b_ub_2 = u - (1 - (x + r)) / (x + r)
        A_ub_2 = np.hstack([np.zeros((m, n)),
                            np.identity(m),
                            np.zeros((m, m)),
                            np.zeros((m,1))])

        b_ub_3 = np.zeros(m)
        A_ub_3 = np.hstack([np.zeros((m, n)),
                            - np.identity(m),
                            np.zeros((m, m)),
                            np.zeros((m,1))])

        b_ub_4 = (1 - (x - r)) / (x - r) - u
        A_ub_4 = np.hstack([np.zeros((m, n)),
                            np.zeros((m, m)),
                            np.identity(m),
                            np.zeros((m, 1))])

        b_ub_5 = np.zeros(m)
        A_ub_5 = np.hstack([np.zeros((m, n)),
                            np.zeros((m, m)),
                            - np.identity(m),
                            np.zeros((m, 1))])

        b_ub_6 = - self.z_lb
        A_ub_6 = np.hstack([-np.identity(n),
                            np.zeros((n, m)),
                            np.zeros((n, m)),
                            np.zeros((n, 1))])


        b_ub_7 = self.z_ub
        A_ub_7 = np.hstack([np.identity(n),
                            np.zeros((n, m)),
                            np.zeros((n, m)),
                            np.zeros((n, 1))])


        A_ub = matrix(np.vstack([
            A_ub_1, A_ub_2, A_ub_3, A_ub_4, A_ub_5, A_ub_6, A_ub_7]))
        b_ub = matrix(np.hstack([
            b_ub_1, b_ub_2, b_ub_3, b_ub_4, b_ub_5, b_ub_6, b_ub_7]))

        return solvers.lp(c, A_ub, b_ub, A_eq, b_eq,
                          solver='glpk',
                          options={'glpk': {'msg_lev':'GLP_MSG_OFF'}})
        
    
    def dDdx_norm_ub(self, cube, nu_lb, nu_ub, lam):

        E = self.E
        x = cube.center
        r = cube.radius

        x_ = x + r
        u = (1 - x_) / x_
        z_arg = (lam.dot(E.T) - nu_lb + nu_ub) / np.inner(self.k, x_) - 1
        z = np.exp(z_arg)
        i = (lam / ((lam > 0) * np.square(np.maximum(self.x_lb, x - r)) +
                    (lam < 0) * np.square(np.minimum(self.x_ub, x + r)))
             - self.w * E.T.dot(z * z_arg / self.b))

        x_ = x - r
        u = (1 - x_) / x_
        z_arg = (lam.dot(E.T) - nu_lb + nu_ub) / np.inner(self.k, x_) - 1
        z = np.exp(z_arg)
        ii = (self.w * E.T.dot(z * z_arg / self.b)
              - lam / ((lam > 0) * np.square(np.minimum(self.x_lb, x + r)) +
                       (lam < 0) * np.square(np.maximum(self.x_ub, x - r))))
        
        return np.max([np.max(i), np.max(ii)])

    
    def iterate_cubes(self, cubes):

        b = self.b
        p_lb = self.p_lb
        p_ub = self.p_ub
        n = self.n
        m = self.m
        
        if self.iter > 1:
            
            cubes = [
                Cube(cube.center + omega * self.radius,
                     self.radius,
                     theta_start=cube.theta_start) for omega in self.omega for cube in cubes
            ]
        
        for cube in cubes:

            # remove cubes that are completely outside [x_lb, x_ub]
            
            if np.any(cube.bottomcorner > self.x_ub):
                cube.remove = True

            # check if feasible for the other cubes
            
            else:

                feasibility_cube_cvx = self.check_feasibility_cube_cvx(cube)
                cube.feasible = True if feasibility_cube_cvx['status'] == 'optimal' else False

                if cube.feasible:

                    delta_plus = np.asarray(feasibility_cube_cvx['x'])[n:n+m,0]
                    delta_min = np.asarray(feasibility_cube_cvx['x'])[n+m:n+2*m,0] 

                    u = (1 - cube.center) / cube.center - delta_plus + delta_min
                    cube.x_delta = 1 / (1 + u)
                    cube.r_delta = cube.radius + norm(cube.center - cube.x_delta, ord=np.inf)
                    
                    if np.any(cube.x_delta - cube.r_delta < 0):
                        cube.rev_ub = np.inf
                        cube.D = 0.0
                    else:
                        with np.errstate(all='ignore'):
                            opt = self.minimize_D(cube)
                        cube.theta_start = opt.x
                        lam = opt.x[:m]
                        nu_lb = np.exp(opt.x[m:n+m])
                        nu_ub = np.exp(opt.x[m+n:])
                        cube.D = opt.fun
                        cube.dDdx_norm_ub = self.dDdx_norm_ub(cube, nu_lb, nu_ub, lam)
                        cube.rev_ub = cube.D + cube.dDdx_norm_ub * cube.r_delta

                    if cube.rev_ub < self.rev_lb:
                        cube.suboptimal = True
                        cube.remove = True
                        
                else:
                    cube.remove = True
                    
        return cubes
    
    
    def minimize_D(self, cube):
        
        success = False
        u = (1 - cube.x_delta) / cube.x_delta
        kx = np.inner(self.k, cube.x_delta)
        count_tries = 0
        theta_start = cube.theta_start
        method = 'BFGS'
            
        while not success:
            
            count_tries += 1
            args = (self.E, u, self.m, self.n, self.z_lb, self.z_ub, kx)
            out = minimize(self.D, theta_start, args=args, jac=self.Dgrad, method=method)
            
            if out.success:
                success = True
            elif count_tries > 10 and out.status != 2:
                exc = (f"lagrange optimization failed at cube.center {1/(1+u)}" +
                       f" and self.radius {self.radius}, opt: \n{out}")
                raise Exception(exc)
                
            if count_tries == 1:
                method = 'BFGS'
            else:
                theta_start = np.random.uniform(-20.0, -50.0, size=2*self.n+self.m)
                
        return out

    
    def bnb(self):
        
        t0 = time.time()
        self.rev_lb = 0.0

        # initialize cubes
        theta_start = np.random.uniform(-20.0, -50.0, size=2*self.n+self.m)
        self.cubes = [Cube(self.x_lb + self.radius, self.radius, theta_start=theta_start)]

        stop = False

        while not stop:

            self.iter += 1
            
            if self.m > 2:
                with Pool() as pool:
                    self.cubes = [cube for cubes in 
                                  pool.map(self.iterate_cubes, np.array_split(self.cubes, pool._processes))
                                  for cube in cubes]
            else:
                self.cubes = self.iterate_cubes(self.cubes)
                
            # self.removed_cubes = self.removed_cubes +\
            #                     [cube for cube in self.cubes if cube.remove]
            self.cubes = [cube for cube in self.cubes if not cube.remove]     
            
            self.rev_lb = np.max(
                [np.max([cube.D for cube in self.cubes]), self.rev_lb]
            )
            rev_ub = np.max([cube.rev_ub for cube in self.cubes])
            opt_gap = self.rev_lb / rev_ub

            if opt_gap  > (1 - self.epsilon):
                self.exit_msg = "Exit because optimality gap is max 1-epsilon."
                stop = True
            elif self.iter == self.max_iter:
                self.exit_msg = f"Exit because maxiter reached, current optimality gap: {opt_gap}."
                stop = True
            else:
                self.radius *= 0.5
                
            # self.xlog['cubes'].append(self.cubes)
            # self.xlog['cubes_to_plot'].append(self.removed_cubes)
            # self.xlog['opt_gap'].append(opt_gap)
            # self.xlog['cube_count'].append(len(self.cubes))

            print(f"iteration: {self.iter}, " +
                  f"cube count: {len(self.cubes)} ({2**self.m * len(self.cubes)}), " +
                  f"opt_gap: {opt_gap:.4f}, rev_lb: {self.rev_lb}")
        
        self.timer = time.time() - t0

In [3]:
def one_run():
    
    m = 3
    max_iter = 5
    a_range = (-4.0, 4.0)
    b_range = (0.001, 0.01)
    epsilon = 0.01
    n = 10
    seed = 1

    bnb = BranchAndBound(
        n=n,
        m=m,
        seed=seed,
        max_iter=max_iter,
        a_range=a_range,
        b_range=b_range,
        epsilon=epsilon,
    )
    bnb.bnb()
    print(bnb.exit_msg)
    print(f"Time elapsed: {bnb.timer:.2f}")
    
%timeit one_run()

iteration: 1, cube count: 1 (8), opt_gap: 0.0000, rev_lb: 0.0
iteration: 2, cube count: 4 (32), opt_gap: 0.1469, rev_lb: 869.5785503882904
iteration: 3, cube count: 32 (256), opt_gap: 0.0000, rev_lb: 898.40274363152
iteration: 4, cube count: 205 (1640), opt_gap: 0.0000, rev_lb: 900.0013411614135
iteration: 5, cube count: 407 (3256), opt_gap: 0.4450, rev_lb: 902.1267204007363
Exit because maxiter reached, current optimality gap: 0.4450402016796101.
Time elapsed: 1.63
iteration: 1, cube count: 1 (8), opt_gap: 0.0000, rev_lb: 0.0
iteration: 2, cube count: 4 (32), opt_gap: 0.1469, rev_lb: 869.5785503882905
iteration: 3, cube count: 32 (256), opt_gap: 0.0000, rev_lb: 898.4027436315199
iteration: 4, cube count: 205 (1640), opt_gap: 0.0000, rev_lb: 900.0013411614137
iteration: 5, cube count: 407 (3256), opt_gap: 0.4450, rev_lb: 902.1267204007362
Exit because maxiter reached, current optimality gap: 0.4450402016796102.
Time elapsed: 1.64
iteration: 1, cube count: 1 (8), opt_gap: 0.0000, rev_lb

# Computation time in n

In [4]:
m = 2
reps = 50
seed = 50
max_iter = np.inf
a_range = (-4.0, 4.0)
b_range = (0.001, 0.01)
epsilon = 0.01
n_range = np.arange(10, 51, 10)

for n in n_range:

    print(f"n: {n}")

    cputime = []
    iterations = []

    for _ in range(reps):

        seed += 1
        bnb = BranchAndBound(
            n=n,
            m=m,
            seed=seed,
            max_iter=max_iter,
            a_range=a_range,
            b_range=b_range,
            epsilon=epsilon,
        )

        bnb.bnb()
        cputime.append(bnb.timer)
        iterations.append(bnb.iter)

    cputime_std_error = np.std(cputime) / np.sqrt(len(cputime))
    cputime = np.mean(cputime)

    iterations_std_error = np.std(iterations) / np.sqrt(len(iterations))
    iterations = np.mean(iterations)

    pd.DataFrame({
        "n": [n],
        "cputime": [cputime],
        "cputime_std_error": [cputime_std_error],
        "iterations": [iterations],
        "iterations_std_error": [iterations_std_error],
    }).to_csv(f"../figs/runtime_in_n{n}_m{m}.csv")

n: 10
iteration: 1, cube count: 1 (4), opt_gap: 0.6734, rev_lb: 496.1897701933706
iteration: 2, cube count: 4 (16), opt_gap: 0.9001, rev_lb: 520.1205316314799
iteration: 3, cube count: 9 (36), opt_gap: 0.9673, rev_lb: 520.9036876016315
iteration: 4, cube count: 11 (44), opt_gap: 0.9871, rev_lb: 521.3273117073632
iteration: 5, cube count: 10 (40), opt_gap: 0.9978, rev_lb: 521.3273117073632
iteration: 1, cube count: 1 (4), opt_gap: 0.0000, rev_lb: 0.0
iteration: 2, cube count: 2 (8), opt_gap: 0.5134, rev_lb: 1203.5228866862433
iteration: 3, cube count: 8 (32), opt_gap: 0.7188, rev_lb: 1256.789723229873
iteration: 4, cube count: 24 (96), opt_gap: 0.9002, rev_lb: 1257.1585236999958
iteration: 5, cube count: 46 (184), opt_gap: 0.9423, rev_lb: 1257.1585236999958
iteration: 6, cube count: 53 (212), opt_gap: 0.9954, rev_lb: 1257.1791993691088
iteration: 1, cube count: 1 (4), opt_gap: 0.6732, rev_lb: 412.50785896127076
iteration: 2, cube count: 4 (16), opt_gap: 0.0000, rev_lb: 412.5078589612707

iteration: 5, cube count: 14 (56), opt_gap: 0.9918, rev_lb: 516.0306715370573
iteration: 1, cube count: 1 (4), opt_gap: 0.6117, rev_lb: 288.37910897631303
iteration: 2, cube count: 4 (16), opt_gap: 0.8744, rev_lb: 306.6080943777653
iteration: 3, cube count: 8 (32), opt_gap: 0.9646, rev_lb: 309.5169821898517
iteration: 4, cube count: 8 (32), opt_gap: 0.9908, rev_lb: 309.788253012115
iteration: 1, cube count: 1 (4), opt_gap: 0.5831, rev_lb: 390.74411319201334
iteration: 2, cube count: 4 (16), opt_gap: 0.6484, rev_lb: 390.74411319201334
iteration: 3, cube count: 8 (32), opt_gap: 0.8545, rev_lb: 390.74411319201334
iteration: 4, cube count: 6 (24), opt_gap: 0.9814, rev_lb: 391.8318901085659
iteration: 5, cube count: 11 (44), opt_gap: 0.9943, rev_lb: 391.9003490336321
iteration: 1, cube count: 1 (4), opt_gap: 0.5725, rev_lb: 499.2908568044235
iteration: 2, cube count: 4 (16), opt_gap: 0.8054, rev_lb: 525.9862728746499
iteration: 3, cube count: 10 (40), opt_gap: 0.9556, rev_lb: 528.8638852564

iteration: 5, cube count: 21 (84), opt_gap: 0.7088, rev_lb: 576.4666337021744
iteration: 6, cube count: 18 (72), opt_gap: 0.9968, rev_lb: 576.6676011355534
iteration: 1, cube count: 1 (4), opt_gap: 0.7490, rev_lb: 302.68827089788704
iteration: 2, cube count: 4 (16), opt_gap: 0.9244, rev_lb: 322.17097142479446
iteration: 3, cube count: 5 (20), opt_gap: 0.9810, rev_lb: 324.94286497804893
iteration: 4, cube count: 7 (28), opt_gap: 0.9938, rev_lb: 324.94286497804893
iteration: 1, cube count: 1 (4), opt_gap: 0.6201, rev_lb: 746.593046393115
iteration: 2, cube count: 4 (16), opt_gap: 0.0000, rev_lb: 746.593046393115
iteration: 3, cube count: 8 (32), opt_gap: 0.0000, rev_lb: 749.0248001833875
iteration: 4, cube count: 9 (36), opt_gap: 0.0000, rev_lb: 749.0248001833875
iteration: 5, cube count: 13 (52), opt_gap: 0.9914, rev_lb: 749.2316929591584
iteration: 1, cube count: 1 (4), opt_gap: 0.5809, rev_lb: 483.91496943969474
iteration: 2, cube count: 4 (16), opt_gap: 0.0000, rev_lb: 483.9149694396

iteration: 5, cube count: 75 (300), opt_gap: 0.0000, rev_lb: 1938.3244904710084
iteration: 6, cube count: 98 (392), opt_gap: 0.9864, rev_lb: 1938.3244904710084
iteration: 7, cube count: 102 (408), opt_gap: 0.9973, rev_lb: 1938.421704360644
iteration: 1, cube count: 1 (4), opt_gap: 0.4324, rev_lb: 732.9968367279112
iteration: 2, cube count: 4 (16), opt_gap: 0.0000, rev_lb: 750.9562216605805
iteration: 3, cube count: 12 (48), opt_gap: 0.0000, rev_lb: 764.1046092099124
iteration: 4, cube count: 17 (68), opt_gap: 0.9772, rev_lb: 764.9035944739802
iteration: 5, cube count: 16 (64), opt_gap: 0.9949, rev_lb: 766.0195972198243
iteration: 1, cube count: 1 (4), opt_gap: 0.5599, rev_lb: 1114.2561214287862
iteration: 2, cube count: 4 (16), opt_gap: 0.7664, rev_lb: 1198.0132784515583
iteration: 3, cube count: 6 (24), opt_gap: 0.9453, rev_lb: 1207.065814202008
iteration: 4, cube count: 8 (32), opt_gap: 0.9868, rev_lb: 1211.9071909007241
iteration: 5, cube count: 7 (28), opt_gap: 0.9970, rev_lb: 1212

iteration: 5, cube count: 13 (52), opt_gap: 0.9895, rev_lb: 1054.4970871099792
iteration: 6, cube count: 15 (60), opt_gap: 0.9973, rev_lb: 1055.2351668812369
iteration: 1, cube count: 1 (4), opt_gap: 0.5576, rev_lb: 1305.8068872077592
iteration: 2, cube count: 4 (16), opt_gap: 0.8295, rev_lb: 1387.524985598796
iteration: 3, cube count: 8 (32), opt_gap: 0.0000, rev_lb: 1428.8448924211502
iteration: 4, cube count: 7 (28), opt_gap: 0.9863, rev_lb: 1429.130554456552
iteration: 5, cube count: 8 (32), opt_gap: 0.9971, rev_lb: 1431.4814547012693
iteration: 1, cube count: 1 (4), opt_gap: 0.0000, rev_lb: 0.0
iteration: 2, cube count: 2 (8), opt_gap: 0.4815, rev_lb: 970.6100865951141
iteration: 3, cube count: 8 (32), opt_gap: 0.5008, rev_lb: 1017.2582798113049
iteration: 4, cube count: 26 (104), opt_gap: 0.4336, rev_lb: 1017.2582798113049
iteration: 5, cube count: 26 (104), opt_gap: 0.7804, rev_lb: 1018.0292523543512
iteration: 6, cube count: 24 (96), opt_gap: 0.9959, rev_lb: 1018.0663429941592


...

iteration: 5, cube count: 14 (56), opt_gap: 0.9930, rev_lb: 977.9056894893673
iteration: 1, cube count: 1 (4), opt_gap: 0.4789, rev_lb: 954.6529106109115
iteration: 2, cube count: 4 (16), opt_gap: 0.6770, rev_lb: 986.6635178006552
iteration: 3, cube count: 14 (56), opt_gap: 0.0000, rev_lb: 986.6635178006552
iteration: 4, cube count: 15 (60), opt_gap: 0.9852, rev_lb: 986.6635178006552
iteration: 5, cube count: 18 (72), opt_gap: 0.9962, rev_lb: 986.8385783567796
iteration: 1, cube count: 1 (4), opt_gap: 0.0000, rev_lb: 0.0
iteration: 2, cube count: 2 (8), opt_gap: 0.5058, rev_lb: 782.7001177055187
iteration: 3, cube count: 8 (32), opt_gap: 0.0000, rev_lb: 782.7001177055187
iteration: 4, cube count: 18 (72), opt_gap: 0.7575, rev_lb: 787.6292745889774
iteration: 5, cube count: 27 (108), opt_gap: 0.9637, rev_lb: 790.2330843764503
iteration: 6, cube count: 20 (80), opt_gap: 0.9919, rev_lb: 790.4693702437603
iteration: 1, cube count: 1 (4), opt_gap: 0.0000, rev_lb: 0.0
iteration: 2, cube coun

iteration: 5, cube count: 20 (80), opt_gap: 0.9862, rev_lb: 1244.602017233315
iteration: 6, cube count: 18 (72), opt_gap: 0.9969, rev_lb: 1245.3692497529946
iteration: 1, cube count: 1 (4), opt_gap: 0.4046, rev_lb: 1257.1682802587752
iteration: 2, cube count: 4 (16), opt_gap: 0.6168, rev_lb: 1349.8056920067527
iteration: 3, cube count: 15 (60), opt_gap: 0.9269, rev_lb: 1367.2973645983398
iteration: 4, cube count: 18 (72), opt_gap: 0.9288, rev_lb: 1370.9973631741636
iteration: 5, cube count: 19 (76), opt_gap: 0.9241, rev_lb: 1371.3328583800196
iteration: 6, cube count: 14 (56), opt_gap: 0.9991, rev_lb: 1371.338867896236
iteration: 1, cube count: 1 (4), opt_gap: 0.4140, rev_lb: 940.8270507749048
iteration: 2, cube count: 4 (16), opt_gap: 0.6144, rev_lb: 1002.5838509022246
iteration: 3, cube count: 16 (64), opt_gap: 0.8845, rev_lb: 1012.7386195137865
iteration: 4, cube count: 11 (44), opt_gap: 0.9851, rev_lb: 1013.8780145656222
iteration: 5, cube count: 11 (44), opt_gap: 0.9966, rev_lb: 1

iteration: 4, cube count: 17 (68), opt_gap: 0.0000, rev_lb: 1462.5214559126466
iteration: 5, cube count: 15 (60), opt_gap: 0.0000, rev_lb: 1462.568752859342
iteration: 6, cube count: 16 (64), opt_gap: 0.9988, rev_lb: 1462.731780022265
iteration: 1, cube count: 1 (4), opt_gap: 0.4635, rev_lb: 1352.38889073834
iteration: 2, cube count: 4 (16), opt_gap: 0.6748, rev_lb: 1432.260824199298
iteration: 3, cube count: 15 (60), opt_gap: 0.9233, rev_lb: 1444.5291846415846
iteration: 4, cube count: 11 (44), opt_gap: 0.9863, rev_lb: 1446.5303070498665
iteration: 5, cube count: 11 (44), opt_gap: 0.9966, rev_lb: 1447.0649395083826
iteration: 1, cube count: 1 (4), opt_gap: 0.4579, rev_lb: 1202.4396774687912
iteration: 2, cube count: 4 (16), opt_gap: 0.6392, rev_lb: 1258.995549795041
iteration: 3, cube count: 16 (64), opt_gap: 0.0000, rev_lb: 1264.1423486359085
iteration: 4, cube count: 10 (40), opt_gap: 0.0000, rev_lb: 1264.2493628680493
iteration: 5, cube count: 15 (60), opt_gap: 0.9960, rev_lb: 1264

iteration: 2, cube count: 4 (16), opt_gap: 0.6073, rev_lb: 1309.891457387127
iteration: 3, cube count: 14 (56), opt_gap: 0.0000, rev_lb: 1325.6467897241241
iteration: 4, cube count: 10 (40), opt_gap: 0.0000, rev_lb: 1325.6613695946912
iteration: 5, cube count: 8 (32), opt_gap: 0.9957, rev_lb: 1325.7059575454027
iteration: 1, cube count: 1 (4), opt_gap: 0.4022, rev_lb: 1711.1093469096377
iteration: 2, cube count: 4 (16), opt_gap: 0.5727, rev_lb: 1784.2605088163064
iteration: 3, cube count: 16 (64), opt_gap: 0.0000, rev_lb: 1788.2885594746485
iteration: 4, cube count: 21 (84), opt_gap: 0.0000, rev_lb: 1788.3871972436614
iteration: 5, cube count: 24 (96), opt_gap: 0.8729, rev_lb: 1788.4122124445967
iteration: 6, cube count: 27 (108), opt_gap: 0.9982, rev_lb: 1788.580905505313
iteration: 1, cube count: 1 (4), opt_gap: 0.5485, rev_lb: 1325.6281146027654
iteration: 2, cube count: 4 (16), opt_gap: 0.6873, rev_lb: 1386.7077771152317
iteration: 3, cube count: 12 (48), opt_gap: 0.9218, rev_lb: 1

iteration: 4, cube count: 9 (36), opt_gap: 0.0000, rev_lb: 1157.4062174384505
iteration: 5, cube count: 11 (44), opt_gap: 0.9964, rev_lb: 1157.4062174384505
iteration: 1, cube count: 1 (4), opt_gap: 0.4333, rev_lb: 1061.0946977475026
iteration: 2, cube count: 4 (16), opt_gap: 0.0000, rev_lb: 1073.2650887591158
iteration: 3, cube count: 12 (48), opt_gap: 0.7853, rev_lb: 1077.2942520710621
iteration: 4, cube count: 27 (108), opt_gap: 0.0000, rev_lb: 1079.1616150780694
iteration: 5, cube count: 22 (88), opt_gap: 0.0000, rev_lb: 1079.649944592694
iteration: 6, cube count: 24 (96), opt_gap: 0.7384, rev_lb: 1079.8548534000388
iteration: 7, cube count: 17 (68), opt_gap: 0.9997, rev_lb: 1079.8838168700872
iteration: 1, cube count: 1 (4), opt_gap: 0.4969, rev_lb: 1528.5992655345487
iteration: 2, cube count: 4 (16), opt_gap: 0.7233, rev_lb: 1646.7325062467676
iteration: 3, cube count: 9 (36), opt_gap: 0.9437, rev_lb: 1666.3583757974013
iteration: 4, cube count: 6 (24), opt_gap: 0.9864, rev_lb: 1

# Computation time in m

In [5]:
n = 50
reps = 50
seed = 50
max_iter = np.inf
a_range = (-4.0, 4.0)
b_range = (0.001, 0.01)
epsilon = 0.01
m_range = [1,2,3,4]

for m in m_range:

    bnbs = []
    print(f"m: {m}")

    cputime = []
    iterations = []

    for _ in range(reps):

        seed += 1
        bnb = BranchAndBound(
            n=n,
            m=m,
            seed=seed,
            max_iter=max_iter,
            a_range=a_range,
            b_range=b_range,
            epsilon=epsilon,
        )

        bnb.bnb()

        cputime.append(bnb.timer)
        iterations.append(bnb.iter)

    cputime_std_error = np.std(cputime) / np.sqrt(len(cputime))
    cputime = np.mean(cputime)

    iterations_std_error = np.std(iterations) / np.sqrt(len(iterations))
    iterations = np.mean(iterations)

    pd.DataFrame({
        "m": [m],
        "cputime": [cputime],
        "cputime_std_error": [cputime_std_error],
        "iterations": [iterations],
        "iterations_std_error": [iterations_std_error],
    }).to_csv(f"figs/runtime_in_m{m}_n{n}.csv")

m: 1
iteration: 1, cube count: 1 (2), opt_gap: 0.4027, rev_lb: 828.473608772519
iteration: 2, cube count: 2 (4), opt_gap: 0.6276, rev_lb: 891.5074274237655
iteration: 3, cube count: 4 (8), opt_gap: 0.9283, rev_lb: 904.5904076465763
iteration: 4, cube count: 3 (6), opt_gap: 0.9844, rev_lb: 907.6260141779119
iteration: 5, cube count: 3 (6), opt_gap: 0.9961, rev_lb: 908.3596206839014
iteration: 1, cube count: 1 (2), opt_gap: 0.3026, rev_lb: 959.740271040773
iteration: 2, cube count: 2 (4), opt_gap: 0.4585, rev_lb: 1032.1209050247023
iteration: 3, cube count: 4 (8), opt_gap: 0.6926, rev_lb: 1047.8553988157114
iteration: 4, cube count: 3 (6), opt_gap: 0.9831, rev_lb: 1051.5949470335572
iteration: 5, cube count: 3 (6), opt_gap: 0.9959, rev_lb: 1052.5096848445926
iteration: 1, cube count: 1 (2), opt_gap: 0.3534, rev_lb: 1054.529037494594
iteration: 2, cube count: 2 (4), opt_gap: 0.5409, rev_lb: 1128.750390945734
iteration: 3, cube count: 4 (8), opt_gap: 0.8176, rev_lb: 1143.9065015775798
iter

iteration: 8, cube count: 961 (15376), opt_gap: 0.8642, rev_lb: 1436.2439721586147
iteration: 9, cube count: 125 (2000), opt_gap: 1.0000, rev_lb: 1436.2506764021705
iteration: 1, cube count: 1 (16), opt_gap: 0.5047, rev_lb: 1446.756808992025
iteration: 2, cube count: 16 (256), opt_gap: 0.0000, rev_lb: 1446.756808992025
iteration: 3, cube count: 144 (2304), opt_gap: 0.0000, rev_lb: 1469.120648178635
iteration: 4, cube count: 863 (13808), opt_gap: 0.0000, rev_lb: 1473.9071379427319
iteration: 5, cube count: 2307 (36912), opt_gap: 0.0000, rev_lb: 1476.0252029571107
iteration: 6, cube count: 4488 (71808), opt_gap: 0.0000, rev_lb: 1476.1805069039954
iteration: 7, cube count: 2823 (45168), opt_gap: 0.9579, rev_lb: 1476.3050451379836
iteration: 8, cube count: 694 (11104), opt_gap: 0.9998, rev_lb: 1476.3307446607419
iteration: 1, cube count: 1 (16), opt_gap: 0.5944, rev_lb: 1303.1718001496147
iteration: 2, cube count: 16 (256), opt_gap: 0.0000, rev_lb: 1303.1718001496147
iteration: 3, cube cou